In [30]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import re 
import json

import numpy as np
# deleted version only 14 states left 
class trafficSet(Dataset):
    def __init__(self,path,train,g_flag=False):
        with open(path,'r') as f:
              testset = json.load(f)
        actions_set = set()
        self.actions = []
        self.rewards = []
        self.data = []
        self.target = []
        self.redun_list = []
        self.redun_dict = {}
        self.states_list = []
        for tset in testset:
            self.actions.append(tset['actions'])
            self.rewards.append(tset['robustness'][0])
            for action in tset['actions']:
                    actions_set.add(action)
        if g_flag is False:
          if train is True:
              self.data = self.actions[:int(len(self.actions)*0.8)]
              self.target = self.rewards[:int(len(self.actions)*0.8)]
          else:
              self.data = self.actions[int(len(self.actions)*0.8):]
              self.target = self.rewards[int(len(self.actions)*0.8):]
        else:
          self.data = self.actions
          self.target = self.rewards
            
        self.actions_list = sorted(list(actions_set))
        self.proxy_actions_list = self.actions_list.copy() + [',','.']
        self.actions_to_index = {self.actions_list[i] : i  for i in range(len(self.actions_list))}
        self.actions_category = []
        self.actions_dict = {}
        self.actions_index = []
        i = 0
        action_string = re.sub(r'[0-9]+', '', self.actions_list[0])
        for action in self.actions_list:
            current_action =  action[:4] + re.sub(r'[0-9]+', '', action[4:])
            if current_action not in self.actions_category:
                self.actions_category = self.actions_category + [current_action]
                self.actions_index.append(i)
                self.actions_dict[current_action] = [action]
            else:
                self.actions_dict[current_action].append(action)
            i = i + 1
        
        self.proxy_actions_indexes= {}
        for i in range(len(self.actions_index)):
            if i != len(self.actions_index) -1 :
                self.proxy_actions_indexes[self.actions_category[i]] = [self.actions_index[i],self.actions_index[i+1]]
            else:
                self.proxy_actions_indexes[self.actions_category[i]] = [self.actions_index[i],len(self.actions_list)]
            i = i + 1
        index = 0
        self.actions_category = []
        for action in self.actions[0]:
          self.actions_category.append(action[:4] + re.sub(r'[0-9]+', '', action[4:]))
        remove_category = []
        remove_action = []
        for category in self.actions_category:
          sindex = self.proxy_actions_indexes[category][0]
          eindex = self.proxy_actions_indexes[category][1]
          if (eindex - sindex) == 1:
            self.redun_list.append(category)
            self.redun_dict[category] = [index ,sindex]
            remove_category.append(category)
            remove_action.append(self.actions_list[sindex])
          index = index + 1        
        for category in remove_category:
          self.actions_category.remove(category)
        for action in remove_action:
          self.actions_list.remove(action)
        self.actions_list = self.actions_list + [',','.'] # bos,eos(.) and pad(,) tokens
        temp_index = []
        temp_category = []
        self.actions_indexes = {}
        i = 0
        last_index = 0
        last_action =  self.actions_list[0][:4] + re.sub(r'[0-9]+', '', self.actions_list[0][:4])
        temp_category.append(last_action)
        for action in self.actions_list:
            current_action =  action[:4] + re.sub(r'[0-9]+', '', action[4:])
            if current_action not in temp_category:
                self.actions_indexes[last_action] = [last_index,i]
                last_action = current_action
                last_index = i

            else:
                continue
            i = i + 1
        self.pad_index = len(self.actions_list) - 2
        self.bos_index = len(self.actions_list) - 1
        self.max_len = len(self.actions_category) + 1

        # self.embeddings = Embedding(len(self.actions_list),emb_dim,self.actions_indexes[','][0])
    def __getitem__(self, index):
        state = self.actions[index]
        state_idx = []
        proxy_idx = []
        remove_idx = -1
        for action in state:
            state_idx.append(self.actions_to_index[action])
        proxy_idx = state_idx.copy()
        count = 0
        for redun in self.redun_list:
          #print(redun)
          state_idx[self.redun_dict[redun][0]] = remove_idx 
        while remove_idx in state_idx:
          state_idx.remove(remove_idx)
        # action = [self.actions_indexes['.'][0]] + action  # add <BOS> for every action
        state = torch.LongTensor(state_idx)
        proxy_state = torch.LongTensor(proxy_idx)
        reward = self.rewards[index]
        reward = np.exp(reward)
        # print(index)
        return proxy_state,state,reward
    def __len__(self):
        return len(self.data)

In [31]:
def gflow2proxy(gflow, redun_list, redun_dict,batch_size,proxy_max_len):
  generated = torch.LongTensor(batch_size, proxy_max_len)
  generated.fill_(-1)
  filled = []
  for redun in redun_list:
    index, action_id = redun_dict[redun]
    generated[:,index] = action_id
    filled.append(index)
  gflow_index = 0
  for i in range(proxy_max_len):
    if i in filled:
      continue
    else:
      generated[:,i] = gflow[:,gflow_index]
      gflow_index = gflow_index + 1
  return generated

In [32]:
gflownet_set = trafficSet("/content/drive/MyDrive/a_testset_for_double_direction.json",train=False)
proxy_state, state, reward = gflownet_set[0]
indexes = gflownet_set.actions_indexes
# print(indexes['time+'])
# print(gflownet_set.actions_category)
# print(gflownet_set.proxy_actions_list[113:137])
print(gflownet_set.redun_list)

['ego+start+lane_position+lane_+.', 'ego+start+speed+.', 'ego+destination+lane_position+lane_+.', 'ego+destination+speed+.', 'npc1+start+lane_position+lane_+.', 'npc1motion++lane_position+lane_+.', 'npc1+destination+lane_position+lane_+.', 'npc1+destination+speed+.', 'npc2+start+lane_position+lane_+.', 'npc2motion++lane_position+lane_+.', 'npc2+destination+lane_position+lane_+.', 'npc2+destination+speed+.', 'npc3+start+lane_position+lane_+.', 'npc3motion++lane_position+lane_+.', 'npc3+destination+lane_position+lane_+.', 'npc3+destination+speed+.', 'npc4+start+lane_position+lane_+.', 'npc4motion++lane_position+lane_+.', 'npc4+destination+lane_position+lane_+.', 'npc4+destination+speed+.', 'npc5+start+lane_position+lane_+.', 'npc5motion++lane_position+lane_+.', 'npc5+destination+lane_position+lane_+.', 'npc5+destination+speed+.']


In [33]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(gflownet_set, 2, shuffle=False,)
data = None
for x in train_dataloader:
  data = x
  break
print(x[0])
print(gflow2proxy(x[1],gflownet_set.redun_list,gflownet_set.redun_dict,2,38))
print(x[1])


tensor([[120, 146, 150, 139,   2,   3,   0,   1,   6,  15,  16,  18,   4,   5,
          29,  38,  39,  49,  27,  28,  52,  60,  61,  71,  50,  51,  74,  82,
          83,  93,  72,  73,  96, 101, 102, 108,  94,  95],
        [124, 142, 149, 137,   2,   3,   0,   1,   6,  15,  16,  18,   4,   5,
          29,  38,  39,  44,  27,  28,  52,  60,  61,  63,  50,  51,  74,  82,
          83,  92,  72,  73,  96, 101, 102, 104,  94,  95]])
tensor([[120, 146, 150, 139,   2,   3,   0,   1,   6,  15,  16,  18,   4,   5,
          29,  38,  39,  49,  27,  28,  52,  60,  61,  71,  50,  51,  74,  82,
          83,  93,  72,  73,  96, 101, 102, 108,  94,  95],
        [124, 142, 149, 137,   2,   3,   0,   1,   6,  15,  16,  18,   4,   5,
          29,  38,  39,  44,  27,  28,  52,  60,  61,  63,  50,  51,  74,  82,
          83,  92,  72,  73,  96, 101, 102, 104,  94,  95]])
tensor([[120, 146, 150, 139,  15,  18,  38,  49,  60,  71,  82,  93, 101, 108],
        [124, 142, 149, 137,  15,  18,  38,  4

In [34]:
import torch
import torch.nn as nn

def Embedding(num_embeddings, embedding_dim, padding_idx=None):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.normal_(m.weight, mean=0, std=embedding_dim ** -0.5)
    if padding_idx is not None:
        nn.init.constant_(m.weight[padding_idx], 0)
    return m
    
class proxy(nn.Module):
    def __init__(self, num_tokens, num_outputs, num_hid,
                 num_layers, max_len=60, dropout=0.1,
                 partition_init=150.0,):
        super(proxy,self).__init__()
        self.input = nn.Linear(num_tokens * max_len, num_hid)
        hidden_layers = []
        for _ in range(num_layers):
            hidden_layers.append(nn.Dropout(dropout))
            hidden_layers.append(nn.ReLU())
            hidden_layers.append(nn.Linear(num_hid, num_hid))
        self.hidden = nn.Sequential(*hidden_layers)
        self.output = nn.Linear(num_hid, num_outputs)
        self.max_len = max_len
        self.num_tokens = num_tokens
        self.emb = Embedding(self.num_tokens,self.num_tokens)

   

    def forward(self, x,  return_all=False, lens=None):
        x = self.emb(x)
        # print(x.shape)
        x = x.reshape(x.size(0),-1)
        # print(x.shape)
        out = self.input(x)
        # out = x.reshape(-1)
        out = self.hidden(out)
        out = self.output(out)
        # print(out.shape)
        out = out.reshape(-1)
        return out



In [35]:
proxy_model =  proxy(num_tokens=154,
                                num_outputs=1,
                                num_hid=1024,
                                num_layers=4, # TODO: add these as hyperparameters?
                                dropout=0.1,
                                max_len=38)
proxy_model.load_state_dict(torch.load("/content/drive/MyDrive/current_w.pth",map_location='cpu')['state_dict'])

<All keys matched successfully>

In [36]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
gflownet_set = trafficSet("/content/drive/MyDrive/a_testset_for_double_direction.json",train=False)
params = AttrDict({
    "n_words": len(gflownet_set.actions_list), 
    "pad_index" : gflownet_set.pad_index, 
    "eos_index" : gflownet_set.bos_index, 
    "bos_index" : gflownet_set.bos_index,
    'max_length': gflownet_set.max_len,
    'actions_index':gflownet_set.actions_indexes,
    'actions_list': gflownet_set.actions_list,
    "actions_category":gflownet_set.actions_category,
    "proxy_actions_list":gflownet_set.proxy_actions_list,
    "emb_dim" : 256, 

})
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
import itertools
import numpy as np

def Embedding(num_embeddings, embedding_dim, padding_idx=None):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.normal_(m.weight, mean=0, std=embedding_dim ** -0.5)
    if padding_idx is not None:
        nn.init.constant_(m.weight[padding_idx], 0)
    return m

def get_padding_masks(slen, lengths):
    """
    Generate hidden states mask
    """
    assert lengths.max().item() <= slen
    bs = lengths.size(0)
    alen = torch.arange(slen, dtype=torch.long, device=lengths.device)
    mask = alen < lengths[:, None]
    # sanity check
    assert mask.size() == (bs, slen)
    return mask

class TransformerModel(nn.Module):

    # params : n_words, eos_index, pad_index, emb_dim

    def __init__(self, params, transformer_layers):
        """
        Transformer model 
        """
        super().__init__()
        # embeddings : one hot is better in this case
        self.embeddings = Embedding(params.n_words, params.emb_dim, padding_idx=params.pad_index)
        # This can be replace by transformer model from torch.nn, huggingface transoformer ...
        self.transformer = transformer_layers

    def forward(self, x, lengths):
        """
        Inputs:
            `x` LongTensor(bs, slen), containing word indices
            `lengths` LongTensor(bs), containing the length of each sentence
        """
        #padding_mask = x != self.pad_index
        #lengths = padding_mask.long().sum(dim=1).to(x.device)
        
        # check inputs
        bs, slen = x.size()
        assert lengths.size(0) == bs
        assert lengths.max().item() <= slen

        # generate masks
        mask = get_padding_masks(slen, lengths)

        # embeddings
        tensor = self.embeddings(x)
        tensor *= mask.unsqueeze(-1).to(tensor.dtype)
        
        # transformer layers
        tensor = self.transformer(tensor)
        
        return tensor
def make_mlp(l, act=nn.LeakyReLU(), tail=[]):
    """makes an MLP with no top layer activation"""
    return nn.Sequential(*(sum(
        [[nn.Linear(i, o)] + ([act] if n < len(l)-2 else [])
         for n, (i, o) in enumerate(zip(l, l[1:]))], []) + tail))

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logZ = torch.zeros((1,)).to(device)
n_hid = 256
n_layers = 2
mlp = make_mlp([params.emb_dim] + [n_hid] * n_layers + [params.n_words]).to(device)
model = TransformerModel(params, mlp).to(device)
P_B = 1 # DAG & sequence generation => tree 


optim = torch.optim.Adam([ {'params':model .parameters(), 'lr':0.0001}, {'params':[logZ], 'lr':0.01} ])
logZ.requires_grad_()

tensor([0.], requires_grad=True)

In [ ]:
a = [1,2,3,4,5,6]
a[2:3] = []
a

[1, 2, 4, 5, 6]

In [38]:
import tqdm

losses_TB = []
zs_TB = []
rewards_TB = []
l1log_TB = []

In [39]:
import torch
import torch.nn.functional as F
a = torch.rand(3,4)
print(a)
print(F.normalize(a))

tensor([[0.0568, 0.8325, 0.0148, 0.5216],
        [0.9980, 0.3752, 0.7383, 0.2324],
        [0.3712, 0.7188, 0.9495, 0.1349]])
tensor([[0.0577, 0.8459, 0.0151, 0.5300],
        [0.7575, 0.2848, 0.5604, 0.1764],
        [0.2958, 0.5729, 0.7568, 0.1076]])


In [ ]:
def softmax_norm(scores,batch_size,vocab_size):
  print(scores)
  score_tensor = torch.zeros(batch_size,vocab_size)
  max_tensor = torch.tensor(torch.max(scores,1).values)
  for i in range(batch_size):
      score_tensor[i,:].fill_(max_tensor[i]) 
  scores = scores - score_tensor
  print(scores)
  return scores

In [40]:
batch_size = 2
max_len = params.max_length + 0
actions_list = params.actions_list
actions_index = params.actions_index
actions_category = params.actions_category
#n_train_steps = 1000
n_train_steps = 10000
for it in tqdm.trange(n_train_steps):
    nan_flag = False
    generated = torch.LongTensor(batch_size, max_len)  # upcoming output
    generated.fill_(params.pad_index)                  # fill upcoming ouput with <PAD>
    generated[:,0].fill_(params.bos_index)             # <BOS> (start token), initial state

    # Length of already generated sequences : 1 because of <BOS>
    #gen_len = (generated != params.pad_index).long().sum(dim=1)
    gen_len = torch.LongTensor(batch_size,).fill_(1) # (batch_size,)
    # 1 (True) if the generation of the sequence is not yet finished, 0 (False) otherwise
    unfinished_sents = gen_len.clone().fill_(1) # (batch_size,)
    # Length of already generated sequences : 1 because of <BOS>
    cur_len = 1 

    # Z_test = model(generated[:,:cur_len].to(device), lengths=gen_len.to(device))
    # #Z_test = Z_test[:,0].squeeze(1).exp().to(device)
    # Z_test = Z_test.sum(dim=1).squeeze(1).exp().to(device)
    # print(Z_test)

    Z = logZ.exp()

    flag = True
    if flag :
        # detached form  of TB
        ll_diff = torch.zeros((batch_size,)).to(device)
        ll_diff += logZ
    else :
        # non-detached form of TB ojective, where we multiply everything before doing the logarithm
        in_probs = torch.ones(batch_size, dtype=torch.float, requires_grad=True).to(device)

    while cur_len < max_len:
        state = generated[:,:cur_len] + 0 # (bs, cur_len)
        tensor = model(state.to(device), lengths=gen_len.to(device)) # (bs, cur_len, vocab_size)
        # print(tensor)
        #scores = tensor[:,0] # (bs, vocab_size) : use last word for prediction
        scores = tensor.sum(dim=1) # (bs, vocab_size)
        
        # fixed length generation
        cur_action_index = actions_index[actions_category[cur_len-1]]
        for index in range(0,cur_action_index[0]):
          scores[:,index] = -1e8
        for index in range(cur_action_index[1],max_len):
          scores[:,index] = -1e8
        
        scores = scores.log_softmax(1)
        
        sample_temperature = 100
        
        # scores = softmax_norm(scores,batch_size,params.n_words)
        # break
        probs = F.softmax(scores / sample_temperature, dim=1)
        # probs = torch.where(torch.isnan(probs),torch.full_like(probs,1e-8),probs)
        try:
          next_words = torch.multinomial(probs, 1).squeeze(1)
        except:
          nan_flag = True
          break      
          
        # update generations / lengths / finished sentences / current length
        generated[:,cur_len] = next_words.cpu() * unfinished_sents + params.pad_index * (1 - unfinished_sents)
        gen_len.add_(unfinished_sents) # add 1 to the length of the unfinished sentences
        unfinished_sents.mul_(next_words.cpu().ne(params.eos_index).long()) # as soon as we generate <EOS>, set unfinished_sents to 0
        cur_len = cur_len + 1

        # loss
        if flag :
            #sample_in_probs = probs.gather(1, next_words.unsqueeze(-1)).squeeze(1)
            #sample_in_probs[unfinished_sents == 0] = 1.
            #ll_diff += sample_in_probs.log()
            
            ll_diff += scores.gather(1, next_words.unsqueeze(-1)).squeeze(1)
        else :
            sample_in_probs = probs.gather(1, next_words.unsqueeze(-1)).squeeze(1)
            sample_in_probs[unfinished_sents == 0] = 1.
            in_probs = in_probs * sample_in_probs
      
        # stop when there is a <EOS> in each sentence, or if we exceed the maximul length
        if unfinished_sents.max() == 0:
            break
    if nan_flag == True:
      nan_flag = False
      continue

    generated = generated.apply_(lambda index : 0 if index == params.pad_index or index == params.eos_index else index)
    #R = reward_function(generated, reward_coef, lambda_, beta).to(device)
    # generated =  [float("".join([str(s_i) for s_i in s])) for s in generated.tolist()]
    # R = reward_function22(generated, reward_coef, lambda_, beta).to(device) 
    generated = gflow2proxy(generated[:,1:],gflownet_set.redun_list,gflownet_set.redun_dict,batch_size,38)
    R = proxy_model(generated)
    
    optim.zero_grad()
    if flag :
        ll_diff -= R.log()
        loss = (ll_diff**2).sum()/batch_size
    else :
        loss = ((Z*in_probs / R).log()**2).sum()/batch_size
    R = R.detach()
    loss.backward()
    optim.step()

    losses_TB.append(loss.item())
    zs_TB.append(Z.item())
    rewards_TB.append(R.mean().cpu())
    
    it = it + batch_size

    if it%100==0: 
        print('\nloss =', np.array(losses_TB[-100:]).mean(), 'Z =', Z.item(), "R =", np.array(rewards_TB[-100:]).mean() )
torch.save(model,"/content/drive/MyDrive/gflownet.pth")
        

  1%|          | 100/10000 [00:08<18:36,  8.87it/s]


loss = 2105.6095830165978 Z = 2.4410769939422607 R = 0.7346751


  2%|▏         | 202/10000 [00:17<11:13, 14.54it/s]


loss = 1981.20121591568 Z = 6.048422813415527 R = 0.7118993


  3%|▎         | 301/10000 [00:24<12:35, 12.84it/s]

...

 81%|████████  | 8100/10000 [10:29<02:43, 11.63it/s]


loss = 316.44311512470244 Z = 6.477559724664422e+16 R = 0.7208471


 82%|████████▏ | 8202/10000 [10:37<01:54, 15.70it/s]


loss = 361.2398410844803 Z = 7.7358150616875e+16 R = 0.7054594


100%|██████████| 10000/10000 [10:56<00:00, 15.22it/s]


In [47]:
samples = []

for it in tqdm.trange(100):
    nan_flag = False
    generated = torch.LongTensor(batch_size, max_len)  # upcoming output
    generated.fill_(params.pad_index)                  # fill upcoming ouput with <PAD>
    generated[:,0].fill_(params.bos_index)             # <BOS> (start token), initial state

    # Length of already generated sequences : 1 because of <BOS>
    #gen_len = (generated != params.pad_index).long().sum(dim=1)
    gen_len = torch.LongTensor(batch_size,).fill_(1) # (batch_size,)
    # 1 (True) if the generation of the sequence is not yet finished, 0 (False) otherwise
    unfinished_sents = gen_len.clone().fill_(1) # (batch_size,)
    # Length of already generated sequences : 1 because of <BOS>
    cur_len = 1 

    while cur_len < max_len:
        state = generated[:,:cur_len] + 0 # (bs, cur_len)
        with torch.no_grad():
            tensor = model(state.to(device), lengths=gen_len.to(device)) # (bs, cur_len, vocab_size)
        #scores = tensor[:,0] # (bs, vocab_size) : use last word for prediction
        scores = tensor.sum(dim=1) # (bs, vocab_size) 
        # fixed length generation
        
        scores = scores.log_softmax(1)
        sample_temperature = 1
        probs = F.softmax(scores / sample_temperature, dim=1)
        #next_words = torch.distributions.categorical.Categorical(probs=probs).sample()
        try:
          next_words = torch.multinomial(probs, 1).squeeze(1)
        except:
          nan_flag = True
          break
        # update generations / lengths / finished sentences / current length

        generated[:,cur_len] = next_words.cpu() * unfinished_sents + params.pad_index * (1 - unfinished_sents)
        gen_len.add_(unfinished_sents) # add 1 to the length of the unfinished sentences
        unfinished_sents.mul_(next_words.cpu().ne(params.eos_index).long()) # as soon as we generate <EOS>, set unfinished_sents to 0
        cur_len = cur_len + 1
      
        # stop when there is a <EOS> in each sentence, or if we exceed the maximul length
        if unfinished_sents.max() == 0:
            break

    #R = reward_function(generated, reward_coef, lambda_, beta).to(device)
    if nan_flag == True:
      nan_flag = False
      continue

    samples.extend(generated)  


100%|██████████| 100/100 [00:00<00:00, 125.81it/s]
